<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study2/blob/main/vibrationTest_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!pip install tensorflow_model_optimization
!pip install h5py

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import time
import psutil
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import to_categorical
import keras.backend as K
import tensorflow_model_optimization as tfmot
import h5py

In [7]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_data'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']

# 데이터 로드 및 전처리 함수 정의
def load_data(base_dir, split):
    X = []
    y = []
    split_dir = os.path.join(base_dir, split)
    for category in categories:
        category_dir = os.path.join(split_dir, category)
        for file in os.listdir(category_dir):
            file_path = os.path.join(category_dir, file)
            data = pd.read_csv(file_path, header=None).values
            data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
            data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
            X.append(data)
            y.append(category)
    return np.array(X), np.array(y)

# 데이터 로드
X_train, y_train = load_data(base_dir, 'train')
X_val, y_val = load_data(base_dir, 'validation')
X_test, y_test = load_data(base_dir, 'test')

# 데이터 차원 변경 (CNN 입력 형식에 맞게)
X_train = np.expand_dims(X_train, axis=-1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# 레이블 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# 원-핫 인코딩
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [17]:
# 기본 CNN 모델 정의
def create_model():
    model = Sequential(name='CNN_Model')
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), name='conv1'))
    model.add(MaxPooling1D(pool_size=2, name='maxpool1'))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', name='conv2'))
    model.add(MaxPooling1D(pool_size=2, name='maxpool2'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(100, activation='relu', name='dense1'))
    model.add(Dropout(0.5, name='dropout'))
    model.add(Dense(len(categories), activation='softmax', name='output'))
    return model

# 모델 생성 및 학습
model = create_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train_categorical, epochs=10, batch_size=32, validation_data=(X_val, y_val_categorical))

Epoch 1/10
219/219 [==============================] - 77s 347ms/step - loss: 1.6074 - accuracy: 0.4862 - val_loss: 0.9466 - val_accuracy: 0.5869
Epoch 2/10
219/219 [==============================] - 76s 347ms/step - loss: 0.7427 - accuracy: 0.6195 - val_loss: 0.3764 - val_accuracy: 0.8427
Epoch 3/10
219/219 [==============================] - 75s 342ms/step - loss: 0.5081 - accuracy: 0.7414 - val_loss: 0.1731 - val_accuracy: 0.9687
Epoch 4/10
219/219 [==============================] - 75s 345ms/step - loss: 0.4042 - accuracy: 0.8041 - val_loss: 0.1072 - val_accuracy: 0.9992
Epoch 5/10
219/219 [==============================] - 76s 346ms/step - loss: 0.3880 - accuracy: 0.8035 - val_loss: 0.1065 - val_accuracy: 1.0000
Epoch 6/10
219/219 [==============================] - 76s 347ms/step - loss: 0.3641 - accuracy: 0.8202 - val_loss: 0.0894 - val_accuracy: 0.9992
Epoch 7/10
219/219 [==============================] - 76s 346ms/step - loss: 0.3366 - accuracy: 0.8347 - val_loss: 0.0940 - val_ac

In [24]:
# 원래 모델 저장
model.save('model.h5')

In [18]:
# 가지치기 함수 정의
def prune_weights(model, pruning_percentage=0.8):
    for layer in model.layers:
        if isinstance(layer, Dense):
            weights, biases = layer.get_weights()

            # 가중치의 절대값 기준으로 임계값 계산
            abs_weights = np.abs(weights)
            threshold = np.percentile(abs_weights, pruning_percentage * 100)

            # 임계값 이하의 가중치를 0으로 설정
            new_weights = np.where(abs_weights < threshold, 0, weights)

            # 가지치기된 가중치로 레이어 설정
            layer.set_weights([new_weights, biases])

    return model

# 가지치기된 모델
pruned_model = prune_weights(model, pruning_percentage=0.8)

In [19]:
# 가지치기된 모델 재훈련
pruned_model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
pruned_model.fit(X_train, y_train_categorical, epochs=5, batch_size=32, validation_data=(X_val, y_val_categorical))

Epoch 1/5
219/219 [==============================] - 76s 342ms/step - loss: 0.2653 - accuracy: 0.8834 - val_loss: 0.0184 - val_accuracy: 1.0000
Epoch 2/5
219/219 [==============================] - 74s 339ms/step - loss: 0.2266 - accuracy: 0.9066 - val_loss: 0.0074 - val_accuracy: 1.0000
Epoch 3/5
219/219 [==============================] - 75s 343ms/step - loss: 0.2078 - accuracy: 0.9219 - val_loss: 0.0061 - val_accuracy: 1.0000
Epoch 4/5
219/219 [==============================] - 75s 341ms/step - loss: 0.2205 - accuracy: 0.9159 - val_loss: 0.0261 - val_accuracy: 0.9992
Epoch 5/5
219/219 [==============================] - 75s 343ms/step - loss: 0.1783 - accuracy: 0.9315 - val_loss: 0.0082 - val_accuracy: 1.0000


In [21]:
# 가지치기된 모델의 가중치 저장 (희소성 적용)
def save_sparse_model(model, file_path):
    with h5py.File(file_path, 'w') as f:
        for layer in model.layers:
            if isinstance(layer, Dense) or isinstance(layer, Conv1D):
                weights, biases = layer.get_weights()
                # 0이 아닌 가중치만 저장
                non_zero_indices = weights != 0
                non_zero_weights = weights[non_zero_indices]
                f.create_dataset(layer.name + '_weights', data=non_zero_weights)
                f.create_dataset(layer.name + '_biases', data=biases)
                f.create_dataset(layer.name + '_non_zero_indices', data=non_zero_indices)
            else:
                # 다른 레이어는 그대로 저장
                weights = layer.get_weights()
                for i, weight in enumerate(weights):
                    f.create_dataset(layer.name + f'_weights_{i}', data=weight)

# 희소성 적용한 모델 저장
save_sparse_model(pruned_model, 'model_after_pruning_sparse.h5')

# 희소성 적용 모델 로드 함수
def load_sparse_model(file_path, original_model):
    with h5py.File(file_path, 'r') as f:
        for layer in original_model.layers:
            if isinstance(layer, Dense) or isinstance(layer, Conv1D):
                non_zero_weights = f[layer.name + '_weights'][:]
                biases = f[layer.name + '_biases'][:]
                non_zero_indices = f[layer.name + '_non_zero_indices'][:]
                original_weights = np.zeros(layer.get_weights()[0].shape)
                original_weights[non_zero_indices] = non_zero_weights
                layer.set_weights([original_weights, biases])
            else:
                weights = []
                i = 0
                while f'{layer.name}_weights_{i}' in f:
                    weights.append(f[layer.name + f'_weights_{i}'][:])
                    i += 1
                layer.set_weights(weights)

# 희소성 적용 모델 로드
model_for_evaluation = create_model()
load_sparse_model('model_after_pruning_sparse.h5', model_for_evaluation)

In [33]:
# 모델 컴파일
model_for_evaluation.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
# 모델 크기 비교
original_model_size = os.path.getsize('model.h5') / (1024 * 1024)
pruned_model_size = os.path.getsize('model_after_pruning.h5') / (1024 * 1024)
sparse_model_size = os.path.getsize('model_after_pruning_sparse.h5') / (1024 * 1024)
print(f"Original Model Size: {original_model_size:.2f} MB")
print(f"Pruned Model Size (without sparsity): {pruned_model_size:.2f} MB")
print(f"Sparse Pruned Model Size: {sparse_model_size:.2f} MB")

Original Model Size: 439.50 MB
Pruned Model Size (without sparsity): 439.50 MB
Sparse Pruned Model Size: 68.73 MB


In [31]:
# 메모리 사용량 확인
process = psutil.Process(os.getpid())
memory_usage_after = process.memory_info().rss / (1024 * 1024)  # 메모리 사용량을 MB 단위로 변환
print(f"Memory Usage (MB): {memory_usage_after:.2f}")

# 가지치기된 모델 추론 시간 측정
start_time = time.time()
y_pred_categorical_after = model_for_evaluation.predict(X_test)
end_time = time.time()
inference_time_after = end_time - start_time

print(f"Inference Time (s) : {inference_time_after:.2f}")

Memory Usage (MB): 12836.93
40/40 [==============================] - 3s 69ms/step
Inference Time (s) : 2.99


In [36]:
# 검증 데이터 평가
val_loss_after, val_accuracy_after = model_for_evaluation.evaluate(X_val, y_val_categorical)
print(f"Validation Loss: {val_loss_after:4f}")
print(f"Validation Accuracy: {val_accuracy_after:4f}")

# 테스트 데이터 평가
test_loss_after, test_accuracy_after = model_for_evaluation.evaluate(X_test, y_test_categorical)
print(f"test Loss: {test_loss_after:4f}")
print(f"testn Accuracy: {test_accuracy_after:4f}")

40/40 [==============================] - 3s 71ms/step - loss: 0.0082 - accuracy: 1.0000
Validation Loss: 0.008229
Validation Accuracy: 1.000000
40/40 [==============================] - 3s 72ms/step - loss: 0.0182 - accuracy: 1.0000
test Loss: 0.018203
testn Accuracy: 1.000000


In [37]:
# 혼동 행렬 및 성능 지표 출력 (테스트 데이터)
y_pred_after = np.argmax(y_pred_categorical_after, axis=1)
conf_matrix_test_after = confusion_matrix(y_test_encoded, y_pred_after)
class_report_test_after = classification_report(y_test_encoded, y_pred_after, target_names=categories)
f1_score_after = f1_score(y_test_encoded, y_pred_after, average='weighted')

# 성능 비교 출력
performance_comparison = {
    'Metric': ['Model Size (MB)', 'Memory Usage (MB)', 'Inference Time (s)', 'Validation Loss', 'Validation Accuracy', 'Test Loss', 'Test Accuracy', 'F1 Score'],
    'After Pruning': [sparse_model_size, memory_usage_after, inference_time_after, val_loss_after, val_accuracy_after, test_loss_after, test_accuracy_after, f1_score_after]
}

import pandas as pd
performance_df = pd.DataFrame(performance_comparison)
print("\nPerformance Comparison:")
print(performance_df)


Performance Comparison:
                Metric  After Pruning
0      Model Size (MB)      68.725109
1    Memory Usage (MB)   12836.925781
2   Inference Time (s)       2.986383
3      Validation Loss       0.008229
4  Validation Accuracy       1.000000
5            Test Loss       0.018203
6        Test Accuracy       1.000000
7             F1 Score       1.000000
